# Congestion Map

The goal is to build a map at a time t of the state of the network.

### Input:
- a speed matrix with for each section the speed

### Output:
- a folium map with the sections in different colors.

### Colors:
- Red: Congestion
- Orange: A lot of traffic
- Green: No congestion
- Gray: No data

### How to compute congestion
- V_t / v_max
- V_t / v_limite

In [1]:
import pandas as pd
import numpy as np

# Maps
import folium
from folium.plugins import FastMarkerCluster
from folium.plugins import MarkerCluster

In [70]:
def get_speed_limit(row):
    road_id = int(row.name.split('_')[0])
    r = roads.find_one({"_id": road_id})
    if 'maxspeed' in r['key']:
        return [item.get('v') for item in r['tag'] if item.get('k')=='maxspeed'][0]
    return False

In [153]:
def compute_congestion(row):
    speed, limit = row[0], row[1]
    if np.isnan(speed):
        return -1
    if limit == False:
        return -1
    try:
        c = float(speed) / float(limit)
    except ValueError:
        c = -1
    return c

In [157]:
def congestion_level(C):
    if C == -1:    # We don't know => gray
        return -1
    if C < 0.25:   # Speed is 25% of limit => red
        return 0
    if C < 0.70:   # Speed is in [25%, 70%] => orange
        return 1
    return 2       # Speed is high enough => green

In [203]:
def roads_by_congestion(congestion, M):
    return list(set([r.split('_')[0] for r in list(M[M['Congestion_level'] == congestion].index.values)]))

def coords_lists(road_list):
    return [roads.find({'_id': int(i)})[0]['loc']['coordinates'] for i in road_list]

def add_color_roads_to_map(f_map, coords, color):
    [folium.PolyLine(locations=[[lo[1],lo[0]] for lo in x], color=color).add_to(f_map) for x in coords]

In [207]:
def get_congestion_map(matrix, save=False, save_name="congestion_map"):
    
    M = pd.DataFrame(matrix)
    
    M['limit'] = M.apply(get_speed_limit ,axis=1)
    print("Routes sans vitesse limite:", (M['limit'] == False).sum())
    
    M['C'] = M.apply(compute_congestion ,axis=1)
    
    M['Congestion_level'] = M['C'].apply(congestion_level)
    
    
    print(M.describe())
    print("\n===========\n")
    print(M['Congestion_level'].value_counts())
    
    green_roads = roads_by_congestion(2, M)
    orange_roads = roads_by_congestion(1, M)
    red_roads = roads_by_congestion(0, M)
    gray_roads = roads_by_congestion(-1, M)

    coords_green = coords_lists(green_roads)
    coords_orange = coords_lists(orange_roads)
    coords_red = coords_lists(red_roads)
    coords_gray = coords_lists(gray_roads)

    
    folium_map = folium.Map(location=[48.111952, -1.679330], zoom_start=12, tiles="OpenStreetMap")
    
    add_color_roads_to_map(folium_map, coords_green, 'green')
    add_color_roads_to_map(folium_map, coords_orange, 'orange')
    add_color_roads_to_map(folium_map, coords_red, 'red')
    add_color_roads_to_map(folium_map, coords_gray, 'gray')

    if save:
        folium_map.save(save_name + '.html')
    
    return folium_map
    

In [115]:
M = pd.DataFrame(matrix)

In [148]:
M['limit'] = M.apply(get_speed_limit ,axis=1)

In [117]:
(M['limit'] == False).sum()

63

- De base: 63 avec False, 395 avec valeur

In [160]:
M

,2018-11-13 17:00:00,limit,C,Congestion_level
matching_road,,,,
466806125_0,1.064516,50,0.021290,0
355353309_0,85.823529,110,0.780214,2
355353311_0,69.600000,110,0.632727,1
355353312_0,95.736842,110,0.870335,2
50072323_0,89.515152,110,0.813774,2
23655056_0,51.090909,90,0.567677,1
30724373_0,93.200000,90,1.035556,2
470091939_0,87.023256,110,0.791121,2
321436568_0,26.965714,90,0.299619,1


In [154]:
M['C'] = M.apply(compute_congestion ,axis=1)

In [185]:
M.describe()

,2018-11-13 17:00:00,C,Congestion_level
count,445.000000,458.000000,458.000000
mean,57.666803,0.434377,1.072052
std,35.511898,0.694772,1.109559
min,0.000000,-1.000000,-1.000000
25%,26.500000,0.221111,0.000000
50%,53.900000,0.686607,1.000000
75%,92.600000,0.960903,2.000000
max,121.333333,1.720000,2.000000


In [159]:
M['Congestion_level'] = M['C'].apply(congestion_level)

In [164]:
M['Congestion_level'].value_counts()

 2    226
 1    112
-1     73
 0     47
Name: Congestion_level, dtype: int64

In [182]:
green_roads = roads_by_congestion(2)
orange_roads = roads_by_congestion(1)
red_roads = roads_by_congestion(0)
gray_roads = roads_by_congestion(-1)

In [183]:
folium_map = folium.Map(location=[48.111952, -1.679330], zoom_start=12, tiles="OpenStreetMap")

coords_green = coords_lists(green_roads)
coords_orange = coords_lists(orange_roads)
coords_red = coords_lists(red_roads)
coords_gray = coords_lists(gray_roads)

add_color_roads_to_map(folium_map, coords_green, 'green')
add_color_roads_to_map(folium_map, coords_orange, 'orange')
add_color_roads_to_map(folium_map, coords_red, 'red')
add_color_roads_to_map(folium_map, coords_gray, 'gray')

#[folium.PolyLine(locations=[[lo[1],lo[0]] for lo in x], color='green').add_to(folium_map) for x in coords_green]
#[folium.PolyLine(locations=[[lo[1],lo[0]] for lo in x], color='orange').add_to(folium_map) for x in coords_orange]
#[folium.PolyLine(locations=[[lo[1],lo[0]] for lo in x], color='red').add_to(folium_map) for x in coords_red]
#[folium.PolyLine(locations=[[lo[1],lo[0]] for lo in x], color='gray').add_to(folium_map) for x in coords_gray]

folium_map


### Testing

In [181]:
import sys
sys.path.append('../scripts/')
sys.path.append('../source/')


from pymongo import MongoClient

client = MongoClient()
congestion = client.congestion
data = congestion.coyote2
osm_rennes = client.osm
roads = osm_rennes.roads


import OsmProcessing
from speed_matrix import SpeedMatrix


In [110]:
sm = SpeedMatrix("congestion", "coyote2", "osm", "roads")
segments=OsmProcessing.getSegments(client['osm'].roads)
roads_ids = segments.index.values.tolist()
speeds = sm.get_speed_matrix(15, roads_ids, 17, 20)

In [111]:
speeds.head()

time,2018-11-13 17:00:00,2018-11-13 17:15:00,2018-11-13 17:30:00,2018-11-13 17:45:00,2018-11-13 18:00:00,2018-11-13 18:15:00,2018-11-13 18:30:00,2018-11-13 18:45:00,2018-11-13 19:00:00,2018-11-13 19:15:00,...,2018-12-11 17:30:00,2018-12-11 17:45:00,2018-12-11 18:00:00,2018-12-11 18:15:00,2018-12-11 18:30:00,2018-12-11 18:45:00,2018-12-11 19:00:00,2018-12-11 19:15:00,2018-12-11 19:30:00,2018-12-11 19:45:00
matching_road,,,,,,,,,,,,,,,,,,,,,
100319558_0,26.0,NaN,NaN,NaN,NaN,NaN,39.0,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100319558_1,33.0,NaN,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100319559_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10062754_0,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,54.0,NaN,NaN,16.0,NaN,NaN,NaN,NaN
10213325_0,NaN,25.0,NaN,NaN,21.0,0.0,12.0,NaN,6.0,0.0,...,21.0,0.0,NaN,14.333333,0.0,0.0,8.666667,NaN,NaN,NaN


In [112]:
per = speeds.isnull().sum(axis=1).sort_values(ascending=True) / speeds.shape[1] * 100
(per < 25).sum()

458

In [113]:
most_percents25 = list(per[(per < 25)].index.values)
print(speeds.shape)
speeds = speeds.loc[most_percents25]
print(speeds.shape)

(6260, 192)
(458, 192)


In [212]:
most_percents25

['466806125_0',
 '355353309_0',
 '355353311_0',
 '355353312_0',
 '50072323_0',
 '23655056_0',
 '30724373_0',
 '470091939_0',
 '321436568_0',
 '355183597_0',
 '582513721_0',
 '197448069_0',
 '5985692_0',
 '27931428_0',
 '582513224_0',
 '8026664_0',
 '353320103_0',
 '376153864_0',
 '4951231_0',
 '376153862_0',
 '24481204_0',
 '50072325_0',
 '4919548_0',
 '345327009_0',
 '48487584_0',
 '25022158_0',
 '27684750_0',
 '111850247_0',
 '181766270_0',
 '217181718_0',
 '321428000_0',
 '197135571_0',
 '111848631_0',
 '4708422_0',
 '30724372_0',
 '124861952_0',
 '4589786_0',
 '4525325_0',
 '355353310_0',
 '416641276_0',
 '293742436_0',
 '180462080_0',
 '564021863_0',
 '514492370_0',
 '51154132_0',
 '197448071_0',
 '97475905_0',
 '582515730_0',
 '15530313_0',
 '400313548_0',
 '217182046_0',
 '217182346_0',
 '222139156_0',
 '124861958_0',
 '180462084_0',
 '470091936_0',
 '115616573_0',
 '27447227_0',
 '25023298_0',
 '30724374_0',
 '23288753_0',
 '5031210_0',
 '4999805_0',
 '416475802_0',
 '322776946

In [114]:
matrix = speeds.iloc[:,0]
matrix.head()

matching_road
466806125_0     1.064516
355353309_0    85.823529
355353311_0    69.600000
355353312_0    95.736842
50072323_0     89.515152
Name: 2018-11-13 17:00:00, dtype: float64

In [211]:
get_congestion_map(speeds.iloc[:,1], save=True, save_name='congestion_2018_11_13-17_00')
get_congestion_map(speeds.iloc[:,2], save=True, save_name='congestion_2018_11_13-17_15')
get_congestion_map(speeds.iloc[:,3], save=True, save_name='congestion_2018_11_13-17_30')
get_congestion_map(speeds.iloc[:,4], save=True, save_name='congestion_2018_11_13-17_45')
get_congestion_map(speeds.iloc[:,5], save=True, save_name='congestion_2018_11_13-18_00')

Routes sans vitesse limite: 63
       2018-11-13 17:15:00           C  Congestion_level
count           444.000000  458.000000        458.000000
mean             57.232003    0.423751          1.041485
std              36.635011    0.694680          1.126770
min               0.000000   -1.000000         -1.000000
25%              24.642857    0.183323          0.000000
50%              52.550000    0.682593          1.000000
75%              92.784873    0.950631          2.000000
max             126.000000    1.282222          2.000000


 2    226
 1     99
-1     74
 0     59
Name: Congestion_level, dtype: int64
Routes sans vitesse limite: 63
       2018-11-13 17:30:00           C  Congestion_level
count           447.000000  458.000000        458.000000
mean             58.863995    0.455190          1.082969
std              36.238186    0.693401          1.117640
min               0.000000   -1.000000         -1.000000
25%              26.583333    0.210042          0.000000
50% 

In [208]:
get_congestion_map(speeds.iloc[:,50])

Routes sans vitesse limite: 63
       2018-11-18 17:30:00           C  Congestion_level
count           390.000000  458.000000        458.000000
mean             65.411785    0.308604          0.868996
std              37.974137    0.858440          1.294780
min               0.000000   -1.000000         -1.000000
25%              30.062500   -1.000000         -1.000000
50%              74.250000    0.717187          2.000000
75%              98.650000    1.004104          2.000000
max             130.000000    1.630000          2.000000


 2    231
-1    126
 1     62
 0     39
Name: Congestion_level, dtype: int64
